In [25]:
import pandas as pd
dataset=pd.read_csv("CKD.csv")
#dataset

In [26]:
dataset=pd.get_dummies(dataset,drop_first=True,dtype=int)
#dataset

In [27]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [28]:
dataset["classification_yes"].value_counts()

classification_yes
1    249
0    150
Name: count, dtype: int64

In [29]:
indep=dataset.drop(["classification_yes"],axis=1)
dep=dataset["classification_yes"]

In [30]:
indep.shape #find rows and cloumns

(399, 27)

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [32]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_grid = {'solver':['newton-cg', 'lbfgs', 'liblinear', 'saga'],
             'penalty':['l2']} 



grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(X_train, y_train) 

Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga']},
             scoring='f1_weighted', verbose=3)

In [34]:
re=grid.cv_results_
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.023738,0.001467,0.004787,0.000400,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.981569,0.981014,0.981217,1.000000,1.000000,0.988760,0.009179,1
1,0.014761,0.005998,0.004189,0.000399,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.981569,0.981014,0.981217,1.000000,1.000000,0.988760,0.009179,1
2,0.004986,0.003025,0.004389,0.000489,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.963284,0.981233,0.962573,0.981217,0.981217,0.973905,0.008965,4
3,0.021741,0.004433,0.004788,0.001324,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.981569,0.981233,0.981217,0.981217,0.981217,0.981291,0.000139,3


In [35]:
grid_predictions = grid.predict(X_test) 

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
print("The confusion Matrix:\n",cm)
# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)
print("The report:\n",clf_report)

The confusion Matrix:
 [[51  0]
 [ 1 81]]
The report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99        51
           1       1.00      0.99      0.99        82

    accuracy                           0.99       133
   macro avg       0.99      0.99      0.99       133
weighted avg       0.99      0.99      0.99       133



In [36]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'penalty': 'l2', 'solver': 'newton-cg'}: 0.9924946382275899


In [37]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

1.0